# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "nya" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead! 
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
g_drive_path = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)
os.environ["gdrive_path"] = g_drive_path
models_path = '%s/models/%s%s_transformer'% (g_drive_path, source_language, target_language)
# model temporary directory for training
model_temp_dir = "/content/drive/My Drive/masakhane/model-temp"
# model permanent storage on the drive
!mkdir -p "$gdrive_path/models/${src}${tgt}_transformer/"

In [ ]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/en-nya-baseline


In [ ]:
#TODO: Skip for retrain
# Install opus-tools
! pip install opustools-pkg 

     |████████████████████████████████| 81kB 5.6MB/s 


In [ ]:
#TODO: Skip for retrain
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-nya.xml.gz not found. The following files are available for downloading:

        ./JW300_latest_xml_en.zip already exists
        ./JW300_latest_xml_nya.zip already exists
 572 KB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-nya.xml.gz

 572 KB Total size
./JW300_latest_xml_en-nya.xml.gz ... 100% of 572 KB
gzip: JW300_latest_xml_en-nya.xml already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [ ]:
# extract the corpus file
! gunzip JW300_latest_xml_$tgt-$src.xml.gz

gzip: JW300_latest_xml_nya-en.xml.gz: No such file or directory


In [ ]:
#TODO: Skip for retrain
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2020-07-12 20:08:28--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en.1’

test.en-any.en.1    100%[===================>] 271.28K  --.-KB/s    in 0.02s   

2020-07-12 20:08:28 (11.8 MB/s) - ‘test.en-any.en.1’ saved [277791/277791]

--2020-07-12 20:08:30--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-nya.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK

In [ ]:
#TODO: Skip for retrain
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [ ]:
#TODO: Skip for retrain
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(10)

Loaded data and skipped 4429/60566 lines since contained in test set.


,source_sentence,target_sentence
0,This publication is not for sale .,Magazini ino si yogulitsa .
1,,Colinga cake ni kuthandiza pa nchito yophunzit...
2,3 Finding the Way,3 Mmene Mungaipezele
3,4 Contentment and Generosity,4 Kukhutila Komanso Kupatsa
4,6 Physical Health and Resilience,6 Thanzi Labwino na Kupilila
5,8 Love,8 Cikondi
6,10 Forgiveness,10 Kukhululuka
7,12 Purpose in Life,12 Colinga ca Moyo
8,14 Hope,14 Ciyembekezo
9,16 Learn More,16 Dziŵani Zambili


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [ ]:
#TODO: Skip for retrain
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#TODO: Skip for retrain
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

     |████████████████████████████████| 51kB 4.1MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144805 sha256=e7a3cd1d3b3a56457aa3239d36cb86516c41ead306b3503e9238370e3113acb6
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
00:00:00.02 0.00 percent complete
00:00:19.79 1.96 percent complete
00:00:39.92 3.92 percent complete
00:00:59.16 5.88 percent complete
00:01:18.63 7.84 percent complete
00:01:38.33 9.79 percent complete
00:01:57.66 11.75 percent complete
00:02:17.39 13.71 percent complete
00:02:36.66 15.67 percent complete


00:02:56.46 17.63 percent complete
00:03:16.09 19.59 percent complete


00:03:35.71 21.55 percent complete
00:03:54.99 23.51 percent complete
00:04:14.15 25.47 percent complete
00:04:33.53 27.42 percent complete
00:04:52.75 29.38 percent complete
00:05:11.77 31.34 percent complete
00:05:30.92 33.30 percent complete
00:05:50.44 35.26 percent complete
00:06:09.47 37.22 percent complete
00:06:29.46 39.18 percent complete
00:06:48.79 41.14 percent complete
00:07:08.00 43.10 percent complete
00:07:27.52 45.05 percent complete
00:07:46.96 47.01 percent complete
00:08:06.48 48.97 percent complete
00:08:26.28 50.93 percent complete
00:08:45.30 52.89 percent complete
00:09:04.50 54.85 percent complete
00:09:23.50 56.81 percent complete
00:09:42.81 58.77 percent complete
00:10:02.45 60.73 percent complete
00:10:21.29 62.68 percent complete
00:10:40.70 64.64 percent complete
00:10:59.94 66.60 percent complete
00:11:19.03 68.56 percent complete
00:11:38.52 70.52 percent complete
00:11:57.42 72.48 percent complete
00:12:16.58 74.44 percent complete
00:12:35.66 76.40 pe

In [ ]:
#TODO: Skip for retrain
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
How Job came to know Jehovah .
Imagine Jacob gently wiping the tears from Joseph’s eyes , comforting him with the same hope that had once comforted Jacob’s grandfather Abraham .
What gathering work is referred to at Matthew 24 : 31 ?
( b ) Why is that ancient event significant for us ?
Consider what is involved in making that amazing feat possible .
Neither are we required to adopt a special posture .
Or what if you lose your job , and you are having difficulty finding another ?
The Bible explains : “ By the trespass of the one man [ Adam ] death ruled as king ” over Adam’s descendants .
The next article will address this concern .
However , more is involved than simply telling your children what is right and what is wrong .

==> train.nya <==
Mmene Yobu anadziŵila Yehova .
Yelekezani kuti mukuona Yakobo atate ake a Yosefe akumupukuta misozi mokoma mtima , kenako akumutonthoza pomulimbikitsa kukhala ndi ciyembekezo cimene cinatonthoza agogo ake a Abulahamu .
Ndi nchito



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [ ]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 2467, done.
remote: Total 2467 (delta 0), reused 0 (delta 0), pack-reused 2467
Receiving objects: 100% (2467/2467), 2.64 MiB | 4.34 MiB/s, done.
Resolving deltas: 100% (1725/1725), done.
Processing /content/joeynmt
     |████████████████████████████████| 61kB 4.5MB/s 
     |████████████████████████████████| 276kB 9.7MB/s 
     |████████████████████████████████| 327kB 19.6MB/s 
     |████████████████████████████████| 17.1MB 205kB/s 
     |████████████████████████████████| 215kB 57.1MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 61kB 10.3MB/s 
     |████████████████████████████████| 747kB 57.5MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=77293 sha256=3c8fae0f346f99b8904b109c79858a5b46d70c5f4a22b0f98cafbd4f266a220e
  Stored in directory: /tmp/pip-ephem-wheel-cache-tty7q62f/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [ ]:
#TODO: Skip for retrain
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path "$gdrive_path/vocab.txt"

# Some output
! echo "BPE Nyanja Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 "$gdrive_path/vocab.txt"  # Herman

bpe.codes.4000	dev.en	     test.bpe.nya    test.en-any.en.1  train.bpe.nya
dev.bpe.en	dev.nya      test.en	     test.nya	       train.en
dev.bpe.nya	test.bpe.en  test.en-any.en  train.bpe.en      train.nya
bpe.codes.4000	dev.nya       test.en		train.bpe.en
dev.bpe.en	models	      test.en-any.en	train.bpe.nya
dev.bpe.nya	test.bpe.en   test.en-any.en.1	train.en
dev.en		test.bpe.nya  test.nya		train.nya
BPE Xhosa Sentences
Izi zin@@ acititsa kuti nd@@ iz@@ idz@@ iŵika monga munthu wos@@ aona mtima .
N’@@ t@@ aphunzila coonadi , ndin@@ al@@ eka n@@ chit@@ oyo ngakhale kuti n’nali kulandila ndalama zambili .
N@@ apeleka citsanzo cabwino kwa ana anga aŵili a@@ amuna , ndipo tsopano n’n@@ a@@ ikidwa pau@@ d@@ indo mumpingo .
Cifukwa co@@ khala wo@@ ona mtima , n@@ ili ndi mbili yabwino kwa anthu amene amat@@ enga mis@@ onkh@@ o ndi ena amene nim@@ acita nawo b@@ iz@@ in@@ esi . ”
Kum@@ en@@ eko anayamba kulambila Mulungu wo@@ ona .
Combined BPE Vocab
ʺ
righte@@
ambuyo
amvu
dif@@
SONG@@
cogn@@

# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [ ]:
def get_last_checkpoint(directory):
  last_checkpoint = ''
  try:
    for filename in os.listdir(directory):
      if 'best' in filename and filename.endswith(".ckpt"):
        return filename
      if not 'best' in filename and filename.endswith(".ckpt"):
          if not last_checkpoint or int(filename.split('.')[0]) > int(last_checkpoint.split('.')[0]):
            last_checkpoint = filename
  except FileNotFoundError as e:
    print('Error Occur ', e)
  return last_checkpoint

In [ ]:
# Copy the created models from the temporary storage to main storage on google drive for persistant storage 
# the content of te folder will be overwrite when you start trainin
!cp -r "/content/drive/My Drive/masakhane/model-temp/"* "$gdrive_path/models/${src}${tgt}_transformer/"
last_checkpoint = get_last_checkpoint(models_path)
print('Last checkpoint :',last_checkpoint)

cp: cannot stat '/content/drive/My Drive/masakhane/model-temp/*': No such file or directory
Last checkpoint : 


In [ ]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{gdrive_path}/train.bpe"
    dev:   "{gdrive_path}/dev.bpe"
    test:  "{gdrive_path}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "{gdrive_path}/vocab.txt"
    trg_vocab: "{gdrive_path}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/{last_checkpoint}" # TODO: uncommented to load a pre-trained model from last checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 50                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "{model_temp_dir}"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language, model_temp_dir=model_temp_dir, last_checkpoint=last_checkpoint)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [ ]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2020-07-12 20:38:52,275 Hello! This is Joey-NMT.
2020-07-12 20:38:52.392375: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-12 20:38:53,554 Total params: 12128768
2020-07-12 20:38:53,556 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_

In [ ]:
# Copy the created models from the temporary storage to main storage on google drive for persistant storage 
!cp -r "/content/drive/My Drive/masakhane/model-temp/"* "$gdrive_path/models/${src}${tgt}_transformer/"

In [ ]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 1000	Loss: 104985.17969	PPL: 69.02498	bleu: 0.72913	LR: 0.00030000	*
Steps: 2000	Loss: 87342.05469	PPL: 33.88090	bleu: 2.09997	LR: 0.00030000	*
Steps: 3000	Loss: 78307.31250	PPL: 23.53396	bleu: 3.31598	LR: 0.00030000	*
Steps: 4000	Loss: 72736.87500	PPL: 18.79830	bleu: 5.24072	LR: 0.00030000	*
Steps: 5000	Loss: 68773.72656	PPL: 16.02127	bleu: 6.90910	LR: 0.00030000	*
Steps: 6000	Loss: 66082.85938	PPL: 14.37346	bleu: 8.21670	LR: 0.00030000	*
Steps: 7000	Loss: 63526.49609	PPL: 12.96529	bleu: 9.07820	LR: 0.00030000	*
Steps: 8000	Loss: 61911.11719	PPL: 12.14747	bleu: 9.82287	LR: 0.00030000	*
Steps: 9000	Loss: 60281.30469	PPL: 11.37461	bleu: 10.86482	LR: 0.00030000	*
Steps: 10000	Loss: 58961.61328	PPL: 10.78499	bleu: 11.17248	LR: 0.00030000	*
Steps: 11000	Loss: 57576.37891	PPL: 10.19894	bleu: 11.80842	LR: 0.00030000	*
Steps: 12000	Loss: 56646.62109	PPL: 9.82355	bleu: 12.65364	LR: 0.00030000	*
Steps: 13000	Loss: 55712.61328	PPL: 9.46036	bleu: 13.00193	LR: 0.00030000	*
Steps: 14000	Loss

In [ ]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"


2020-07-12 22:31:35,573 Hello! This is Joey-NMT.
/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
2020-07-12 22:32:01,727  dev bleu:  17.72 [Beam search decoding with beam size = 5 and alpha = 1.0]
2020-07-12 22:32:46,046 test bleu:  30.09 [Beam search decoding with beam size = 5 and alpha = 1.0]
